In [1]:
%pwd

'c:\\Users\\Dell\\genAI Projects\\MedBot\\research'

In [2]:
import os 
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\Dell\\genAI Projects\\MedBot'

In [4]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
# extract data from pdf
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [6]:
extracted_data = load_pdf_file(data='Data')

In [7]:
#extracted_data

In [8]:
#splitting data into chunks

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [9]:
text_chunks = text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5859


In [10]:
#text_chunks

In [21]:
from langchain.embeddings import HuggingFaceEmbeddings

In [24]:
# download huggingface embedding model

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name = 'all-MiniLM-L6-v2')
    return embeddings

In [25]:
embeddings = download_hugging_face_embeddings()

In [26]:
query_result = embeddings.embed_query("Hello world")
print("Length",len(query_result))

Length 384
